<a href="https://colab.research.google.com/github/skochar1/Fair-Clustering-Unsupervised-Learning/blob/main/AAAIWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section I: Setting up the Adult Data

In [ ]:
! pip install kaggle --upgrade

In [ ]:
! mkdir ~/.kaggle/

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d wenruliu/adult-income-dataset

  0% 0.00/652k [00:00<?, ?B/s]
100% 652k/652k [00:00<00:00, 105MB/s]


In [ ]:
! mkdir dataset

In [ ]:
! unzip adult-income-dataset.zip -d dataset

Archive:  adult-income-dataset.zip
  inflating: dataset/adult.csv       


In [ ]:
! ls dataset/

adult.csv


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

In [ ]:
np.random.seed(42)

df = pd.read_csv('dataset/adult.csv')
df = df.sample(frac = 1) # shuffle df rows

In [ ]:
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
7762,56,Private,33115,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,0,40,United-States,<=50K
23881,25,Private,112847,HS-grad,9,Married-civ-spouse,Transport-moving,Own-child,Other,Male,0,0,40,United-States,<=50K
30507,43,Private,170525,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,14344,0,40,United-States,>50K
28911,32,Private,186788,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,<=50K
19484,39,Private,277886,Bachelors,13,Married-civ-spouse,Sales,Wife,White,Female,0,0,30,United-States,<=50K


# Implementing the Unfair Algorithms

K-center algorithm (2.2)

Reference source: Geeks for Geeks 

In [ ]:
def max_ind(dist, n):
    start = 0
    for i in range(n):
        if (dist[i] > dist[start]):
            start = i
    return start
 
def kcenter(n, dists_list, k):
    '''
    Args: 
        n: number of nodes
        dists_list: distances between every pair of nodes
        k: number of clusters
    
    ''' 
    dist = [0]*n
    centers = []
    for i in range(n):
        dist[i] = 10**9
    max_i = 0
    for i in range(k):
        centers.append(max_i)
        for j in range(n):
            dist[j] = min(dist[j], dists_list[max][j])
        max = max_ind(dist, n)
    print(dist[max])
    for i in centers:
        print(i, end = " ")

K-median algorithm (9.2)

Reference source: https://github.com/hmofrad/clustering/blob/master/kmedians.py

In [ ]:
import numpy as np
import os

FILE = '' #FILLER
# Debug level
VERBOSE = False

def read(FILE):
    if os.path.isfile(FILE):
        file = open(FILE, 'r')
        lines = tuple(file)
        file.close()
        data = []
        for line in lines:
            data.append(line.rstrip().split(","))
            if VERBOSE:
                print(data[-1])
            else:
                print(FILE, 'does not exist')
            exit(0)
    
    data = np.array(data)
    x = data[:,0:-1]
    x = x.astype(np.float)
    
    y = np.zeros(len(data))
    uniq = np.unique(data[:,-1])
    for i in range(0,len(uniq)):
        idx = (data[:,-1] == uniq[i])
        if any(idx):
            y[idx] = i
    return(x, y)

[x, y] = read(FILE)

K-mediods algorithm

Reference: https://github.com/hmofrad/clustering/blob/master/kmedoids.py

In [ ]:
import numpy as np
np.random.seed()

[x, y] = read(FILE)

# Initliaze parameters
[n, d] = np.shape(x)  # [#samples, #dimensions]
k = len(np.unique(y)) #  #clusters


# Calculate pairwise Euclidean distance
dist = np.zeros((n,n))
for i in range(n):
    dist[i,:] = np.sqrt(np.sum(np.power(x[i,:] - x,2), axis=1))

c = np.zeros(n) # Cluster assignment
# Randomly pick k samples as
# clusters center
a = np.arange(n)
np.random.shuffle(a)
a = np.sort(a[0:k]) # Clusters center
a_t = np.copy(a)    # Copy of cluster centers

imax = 100
for i in range(imax):
   # Calculate cluster assignment 
   c = np.argmin(dist[:,a], axis=1)
   # Update cluster assignment
   for j in range(k):
        aa = np.arange(n)
        idx = aa[c == j] # Current cluster
        
        idxx = np.argmin(np.mean(dist[np.ix_(idx,idx)],axis=1))
        a_t[j] = idx[idxx]
        np.sort(a_t)
        if np.array_equal(a, a_t):
            break
        a = np.copy(a_t)